## ewf-wfp-02-02-03 - Snow Cover Characterization Anomalies

Snow Cover Characterization Anomalies

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Snow Cover Characterization Anomalies'),
                ('abstract', 'Snow Cover Characterization Anomalies'),
                ('id', 'ewf-wfp-02-02-03')])

### <a name="parameter">Parameter Definition 

In [ ]:
indexAggCat = dict([('id', 'indexAggCat'),
                    ('value', 'better-wfp-02-02-01'),
                    ('title', 'indexAggCat'),
                    ('abstract', 'index to access catalog of aggregated land surface temperature time series'),
                    ('minOccurs', '1')])

In [ ]:
apikeyAggCat = dict([('id', 'apikeyAggCat'),
                     ('value', ''),
                     ('title', 'apikeyAggCat'),
                     ('abstract', 'apikey to access indexAggCat catalog'),
                     ('minOccurs', '1')])

In [ ]:
indexLtaCat = dict([('id', 'indexLtaCat'),
                    ('value', 'better-wfp-02-02-02'),
                    ('title', 'indexLtaCat'),
                    ('abstract', 'index to access catalog of aggregated land surface temperature time series'),
                    ('minOccurs', '1')])

In [ ]:
apikeyLtaCat = dict([('id', 'apikeyLtaCat'),
                     ('value', ''),
                     ('title', 'apikeyLtaCat'),
                     ('abstract', 'apikey to access indexAggCat catalog'),
                     ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the MDOIS stack of products' identifiers

In [ ]:
#input_identifiers = ('2FBBF58C17C3124FDDCFF4BC8E7C985FFFD17DBE', '0C63C452BB93FCFAF834FC1724E00364B84B92A7')
input_identifiers = ('4B4BF219C839E7428A0828B3AB0BCA82311D8511','04843237511A009385116E9C13DCDB2A09B33A97')

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references = ('https://catalog.terradue.com/better-wfp-02-02-01/search?uid={}'.format(input_identifiers[0]), 'https://catalog.terradue.com/better-wfp-02-02-02/search?uid={}'.format(input_identifiers[1])) 

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = "/workspace/dev/ewf-wfp-02-02-03/src/main/app-resources/notebook/libexec"

#### Aux folders

In [ ]:
output_folder = ''

In [ ]:
temp_folder = 'temp'

#### Import Modules

In [ ]:
import os
import shutil

import cioppy
import datetime as dt

import pandas as pd
import geopandas as gpd

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import pdb
ciop = cioppy.Cioppy()

#### Auxiliary vars

In [ ]:
check_results = True

#### Auxiliary methods

In [ ]:
def get_input_metadata (input_refs):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        if 'LTA_' in product_ref:
            apikey = apikeyLtaCat
            indexCat = indexLtaCat
        else:
            apikey = apikeyAggCat
            indexCat = indexAggCat
        
        # since the search is by identifier 
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,title,startdate,enddate,wkt',creds='{}:{}'.format(indexCat['value'],apikey['value']))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata

def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e) 
    
def get_metadata(filepath):
        
    #pdb.set_trace()
    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type


def get_matrix_list(image_list):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
        
        print(type(product_array))
    return mat_list


def calc_anomaly(agg_file, LTA_file):
    
    
    if agg_file and LTA_file:
        
        
        projection, geotransform, no_data_value, data_type = get_metadata(agg_file)
        
        
        agg_and_LTA = get_matrix_list([agg_file, LTA_file])
        print('Aggregation and LTA converted to matrices')
        
    
        if 'uint' in str(agg_and_LTA[0].dtype) or 'uint' in str(agg_and_LTA[1].dtype):
            agg_and_LTA[0] = np.int16(agg_and_LTA[0])
            agg_and_LTA[1] = np.int16(agg_and_LTA[1])
            
        anomaly_values = agg_and_LTA[0] - agg_and_LTA[1]
        
        anomaly_values[np.logical_or(agg_and_LTA[0] == no_data_value, agg_and_LTA[1] == no_data_value)] = no_data_value

        
        return anomaly_values, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None
   


def get_anom_dates (agg, LTA, first_year_agg, no_data_value):

    if agg == no_data_value or LTA == no_data_value:
        return no_data_value
    
    aug_1st = dt.date(first_year_agg, 8, 1).timetuple().tm_yday # 1st aug
    ndays_year = dt.date(first_year_agg, 12, 31).timetuple().tm_yday ## days of the first year

    if agg < aug_1st and LTA < aug_1st:
    
        anom = agg - LTA
    
    elif agg >= aug_1st and LTA >= aug_1st:
    
        anom = agg - LTA
    
    elif agg < aug_1st and LTA >= aug_1st:
    
        d1 = ndays_year - LTA
        d2 = agg
        anom = d1 + d2
    
    elif agg >= aug_1st and LTA < aug_1st:
    
        d1 = ndays_year - agg
        d2 = LTA
        anom = -(d1 + d2)
    
    return anom


def calc_anomaly_dates(agg_file, LTA_file, first_year_agg):
    
    if agg_file and LTA_file:
        
        
        agg_and_LTA = get_matrix_list([agg_file, LTA_file])
        print('Aggregation and LTA converted to matrices')
        
        projection, geotransform, no_data_value, data_type = get_metadata(agg_file)
        
        
        anom_dates_calculator = np.vectorize(get_anom_dates)
            
        anomaly_values = anom_dates_calculator(agg_and_LTA[0], agg_and_LTA[1], first_year_agg, no_data_value)
        
        
        return anomaly_values, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None, None, None
    
    

def write_output_image(filepath, output_matrix, image_format, data_format, mask=None, output_projection=None, output_geotransform=None, no_data_value=None):
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    if mask is not None and mask is not 0:
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
        if no_data_value is not None:
            output_matrix[mask > 0] = no_data_value
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
    
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)

    

def write_anomaly_output(anomaly, output_folder, product_name, first_date, last_date, lta_start_year, lta_end_year, mask_no_value, roi_name, projection, geo_transform, no_data_value):
    
    filename = os.path.join(output_folder, '_'.join(['Anomaly', product_name, roi_name, first_date.split('_')[0], last_date.split('_')[0], 'LTA', str(lta_start_year), str(lta_end_year)]) + '.tif')
    
    write_output_image(filename, anomaly, 'GTiff', gdal.GDT_Float32, mask_no_value, projection, geo_transform, no_data_value)
    return filename

def get_formatted_date(date_str):
    date = datetime.datetime.strftime(date_str, '%Y-%m-%dT00:00:00Z')
    return dat

def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

#### Auxiliary folders

In [ ]:
#Create folders
#if not os.path.isdir(data_path):
#    os.mkdir(data_path)

if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

In [ ]:
input_metadata = get_input_metadata(input_references)
input_metadata

#### Workflow

In [ ]:
if isinstance(input_identifiers, str):
    input_identifiers = [input_identifiers]


enclosure = list(input_metadata['enclosure'])
filepath_agg = os.path.join(data_path, os.path.basename(enclosure[0]).split('?')[0]) 
filepath_LTA = os.path.join(data_path, os.path.basename(enclosure[1]).split('?')[0])

# list of files
print(filepath_agg)
print(filepath_LTA)

In [ ]:
# get metadata from filenames

file_name_elements = os.path.basename(filepath_agg).split('.')[0].split('_')
print(file_name_elements)



first_date = file_name_elements[-2]
last_date = file_name_elements[-1]
agg_type = '_'.join(file_name_elements[-5:-3])
name_of_region = file_name_elements[-3]


file_name_elements = os.path.basename(filepath_LTA).split('.')[0].split('_')


lta_start_year = file_name_elements[-2]
lta_end_year = file_name_elements[-1]

prod_type = file_name_elements[1] + '_' + file_name_elements[-4]


print(filepath_agg)
print(filepath_LTA)

if 'SeasonDate' in prod_type:
    anomaly_values, projection, geotransform, no_data_value, data_type = calc_anomaly_dates(filepath_agg, filepath_LTA, int(first_date))
else:
    anomaly_values, projection, geotransform, no_data_value, data_type = calc_anomaly(filepath_agg, filepath_LTA)


filename = write_anomaly_output(anomaly_values, output_folder, prod_type, first_date, last_date, lta_start_year, lta_end_year, None, name_of_region, projection, geotransform, no_data_value)


In [ ]:
startdate = input_metadata['startdate'].iloc[0].strftime('%Y-%m-%dT00:00:00Z')
enddate = input_metadata['enddate'].iloc[0].strftime('%Y-%m-%dT00:00:00Z')
wkt = input_metadata['wkt'].iloc[0]

write_properties_file(filename, startdate, enddate, wkt)

In [ ]:
if check_results:

    import matplotlib
    import matplotlib.pyplot as plt

    fig = plt.figure()
    plt.imshow(anomaly_values)
    plt.show()

#### Remove temporay files and folders

In [ ]:
rm_cfolder(temp_folder)

os.rmdir(temp_folder)